In [ ]:
!pip install catboost

### Load model and test local inference

Below model is from this example: https://github.com/catboost/tutorials/blob/master/python_tutorial_with_tasks.ipynb

In [ ]:
from catboost import CatBoostClassifier, Pool as CatboostPool, cv
import os
import pandas
model_file = CatBoostClassifier()
model_file = model_file.load_model("catboost_model.bin")
df = pandas.read_csv("val_data.csv")


print(model_file.get_params())
print(model_file.random_seed_)
print(model_file.learning_rate_)
df

In [ ]:
import pandas as pd
import io
import json
df2 = pd.read_csv(io.StringIO(df.to_csv(index=False)))
df2
out = model_file.predict_proba(df2)
val = [dict(enumerate(out.flatten(), 1))]
json_value = json.dumps(val, indent=2).encode("utf-8")
json.loads(json_value)

### Create 1000 copies of CatBoost model

In [ ]:
import sagemaker

sess = sagemaker.Session()
s3_bucket=sess.default_bucket()  # Replace with your own bucket name if needed
print(s3_bucket)

In [ ]:
import boto3

s3 = boto3.client('s3')
for i in range(0,1000):
    with open("catboost-model.tar.gz", "rb") as f:
        s3.upload_fileobj(f, s3_bucket, "catboost/catboost-model-{}.tar.gz".format(i))


In [ ]:
!aws s3 ls s3://$s3_bucket/catboost/

### SageMaker endpoint deployment

In [ ]:
%%sh

# The name of our algorithm
algorithm_name=catboost-sagemaker-multimodel

cd container

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-east-1}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email)

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build -q -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

In [ ]:
import boto3
from sagemaker import get_execution_role

sm_client = boto3.client(service_name='sagemaker')
runtime_sm_client = boto3.client(service_name='sagemaker-runtime')

account_id = boto3.client('sts').get_caller_identity()['Account']
region = boto3.Session().region_name

role = get_execution_role()

In [ ]:
from time import gmtime, strftime

model_name = 'catboost-multimodel-' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
model_url = 's3://{}/catboost/'.format(s3_bucket) ## MODEL S3 URL
container = '{}.dkr.ecr.{}.amazonaws.com/catboost-sagemaker-multimodel:latest'.format(account_id, region)
instance_type = 'ml.c5d.18xlarge'

print('Model name: ' + model_name)
print('Model data Url: ' + model_url)
print('Container image: ' + container)

container = {
    'Image': container,
    'ModelDataUrl': model_url,
    'Mode': 'MultiModel'
}

create_model_response = sm_client.create_model(
    ModelName = model_name,
    ExecutionRoleArn = role,
    Containers = [container])

print("Model Arn: " + create_model_response['ModelArn'])

In [ ]:
endpoint_config_name = 'catboost-multimodel-config' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print('Endpoint config name: ' + endpoint_config_name)

create_endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName = endpoint_config_name,
    ProductionVariants=[{
        'InstanceType': instance_type,
        'InitialInstanceCount': 1,
        'InitialVariantWeight': 1,
        'ModelName': model_name,
        'VariantName': 'AllTraffic'}])

print("Endpoint config Arn: " + create_endpoint_config_response['EndpointConfigArn'])

In [ ]:
%%time

import time

endpoint_name = 'catboost-multimodel-endpoint' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print('Endpoint name: ' + endpoint_name)

create_endpoint_response = sm_client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name)
print('Endpoint Arn: ' + create_endpoint_response['EndpointArn'])

resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp['EndpointStatus']
print("Endpoint Status: " + status)

print('Waiting for {} endpoint to be in service...'.format(endpoint_name))
waiter = sm_client.get_waiter('endpoint_in_service')
waiter.wait(EndpointName=endpoint_name)

In [ ]:
response = runtime_sm_client.invoke_endpoint(
        EndpointName=endpoint_name,
        TargetModel="catboost-model-1.tar.gz",
        Body=df.to_csv(index=False))

In [ ]:
json.loads(response['Body'].read().decode('utf-8'))